In [8]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

In [2]:
ratings_df = pd.read_csv('../data/ml-latest-small/ratings.csv')
ratings_df_sorted = ratings_df.sort_values(['userId','movieId'])
ratings_df_sorted.tail()

,userId,movieId,rating,timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [62]:
R = ratings_df_sorted.pivot(index='userId', columns='movieId', values='rating')
R.tail()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
R_imputed = R.fillna(0)

In [94]:
model = NMF(n_components=20, init='random', random_state=15, max_iter=1000)

model.fit(R_imputed)

Q = model.components_  

P = model.transform(R_imputed)  

print(model.reconstruction_err_)

875.4199592106912


In [66]:
def nmf_recommender(user_id):
    """
    return k highest rated unseen movies as predicted by non negative matrix factorization
    """
        
    ratings_user = R_imputed.loc[[user_id]]
    query = ratings_user.values
    
    p_user = model.transform(query)
    r_hat_user = model.inverse_transform(p_user)
    
    user_boolean_series = R.loc[[user_id]].isna().T    
    not_seen_films = user_boolean_series.index[user_boolean_series[user_id]].tolist()
    
    output = (R_imputed.loc[user_id,not_seen_films]).idxmax()
        
    
    return output

In [92]:
nmf_recommender(88)

1

In [19]:
R_imputed.loc[[1]].values

array([[4.        , 3.43181818, 4.        , ..., 3.5       , 3.5       ,
        4.        ]])